In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import torch

from inference.train import train
from inference.plannar import NormalizingFlow
from simulate.simulate import simulate
from plot.plot_hist_loss import plot_loss
from plot.plot_posterior import plot_posterior
from data.get_experimental_data import get_experimental_data

In [2]:
%config InlineBackend.figure_format = "retina"
sns.set()

In [3]:
data = get_experimental_data()
pd.DataFrame(data)

number of user 53
number of items 1998
total number of observations (excluding first presentation) 70618
minimum number of observation for a single user 1285
maximum number of observation for a single user 1404


,u,w,x,r,y
0,0,1537,"(tensor(61.9800, dtype=torch.float64),)","(tensor(0., dtype=torch.float64),)","(tensor(1., dtype=torch.float64),)"
1,0,1967,"(tensor(116.7440, dtype=torch.float64),)","(tensor(0., dtype=torch.float64),)","(tensor(1., dtype=torch.float64),)"
2,0,1488,"(tensor(115.6600, dtype=torch.float64),)","(tensor(0., dtype=torch.float64),)","(tensor(1., dtype=torch.float64),)"
3,0,1198,"(tensor(138.5150, dtype=torch.float64),)","(tensor(0., dtype=torch.float64),)","(tensor(1., dtype=torch.float64),)"
4,0,563,"(tensor(154.0350, dtype=torch.float64),)","(tensor(0., dtype=torch.float64),)","(tensor(1., dtype=torch.float64),)"
...,...,...,...,...,...
70613,52,296,"(tensor(410.1850, dtype=torch.float64),)","(tensor(9., dtype=torch.float64),)","(tensor(1., dtype=torch.float64),)"
70614,52,606,"(tensor(418.8100, dtype=torch.float64),)","(tensor(11., dtype=torch.float64),)","(tensor(1., dtype=torch.float64),)"
70615,52,807,"(tensor(407.3150, dtype=torch.float64),)","(tensor(14., dtype=torch.float64),)","(tensor(1., dtype=torch.float64),)"
70616,52,538,"(tensor(374.8170, dtype=torch.float64),)","(tensor(10., dtype=torch.float64),)","(tensor(1., dtype=torch.float64),)"


In [4]:
df_param = pd.read_csv("bkp/param_exp_data.csv", index_col=0)
df_param

,mu1,sigma_u1,sigma_w1,mu2,sigma_u2,sigma_w2
unconstrained,-5.661843,1.844262,1.616331,-0.723793,1.720237,1.218074


In [5]:
def generate_param(n_u, n_w, df_param):
    
    mu = np.array([df_param.loc["unconstrained", f"mu{i}"] for i in range(1, 3)])
    sg_u = np.array([df_param.loc["unconstrained", f"sigma_u{i}"] for i in range(1, 3)])
    sg_w = np.array([df_param.loc["unconstrained", f"sigma_w{i}"] for i in range(1, 3)])
    
    Zu = np.random.normal(np.zeros(2), sg_u, size=(n_u, 2))
    Zw = np.random.normal(np.zeros(2), sg_w, size=(n_w, 2))
    
    return mu, Zu, Zw

In [ ]:
class Leitner:

    def __init__(self, env,  delay_factor=2, delay_min=4):

        self.n_item = env.n_item

        self.delay_factor = delay_factor
        self.delay_min = delay_min

        self.box = np.full(self.n_item, -1)
        self.due = np.full(self.n_item, -1)

    def update_box_and_due_time(self, last_idx,
                                last_was_success, last_time_reply):

        if last_was_success:
            self.box[last_idx] += 1
        else:
            self.box[last_idx] = \
                max(0, self.box[last_idx] - 1)

        delay = self.delay_factor ** self.box[last_idx]
        # Delay is 1, 2, 4, 8, 16, 32, 64, 128, 256, 512 ...
        self.due[last_idx] = \
            last_time_reply + self.delay_min * delay

    def _pickup_item(self, now):

        seen = np.argwhere(np.asarray(self.box) >= 0).flatten()
        n_seen = len(seen)

        if n_seen == self.n_item:
            return np.argmin(self.due)

        else:
            seen__due = np.asarray(self.due)[seen]
            seen__is_due = np.asarray(seen__due) <= now
            if np.sum(seen__is_due):
                seen_and_is_due__due = seen__due[seen__is_due]

                return seen[seen__is_due][np.argmin(seen_and_is_due__due)]
            else:
                return self._pickup_new()

    def _pickup_new(self):
        return np.argmin(self.box)

    def ask(self, now, last_was_success, last_time_reply, idx_last_q):

        if idx_last_q is None:
            item_idx = self._pickup_new()

        else:

            self.update_box_and_due_time(
                last_idx=idx_last_q,
                last_was_success=last_was_success,
                last_time_reply=last_time_reply)
            item_idx = self._pickup_item(now)

        return item_idx

    def act(self, obs):

        """
        Adaptation for RL env
        """

        # TODO: Adapt the code

In [20]:
class DummyEnv:
    def __init__(self, n_item):
        self.n_item = n_item


def simulate_human_like(
        mu, Zu, Zw,
        n_u=53,
        n_w=1998,
        n_ss=6,
        n_iter_per_session=100,
        use_torch=False,
        seed=0):

    np.random.seed(seed)
    
    n_obs = n_iter_per_session*n_ss*n_u
    
    u = np.zeros(n_obs)
    w = np.tile(n_obs)
    r = np.zeros(n_obs)
    x = np.zeros(n_obs)
    y = np.zeros(n_obs)
    
    for user in range(n_u):
        
        env = DummyEnv(n_item=n_w)
        teacher = Leitner(env)
        
        last_was_success = None
        last_time_reply = None
        idx_last_q = None
        
        for session in range(n_ss):
            for iteration in range(n_iter_per_session):
                teacher.ask(now=t)

    a = np.exp(Z[:, 0])
    b = expit(Z[:, 1])
    neg_rate = - a * x * (1 - b) ** r
    p = np.exp(neg_rate)
    y[:] = p > rd

    data = {'u': u, 'w': w,
            'x': x, 'r': r,
            'y': y}

    sg_u_smp = np.std(Zu, axis=0)
    sg_w_smp = np.std(Zw, axis=0)
    mu_smp = np.mean(Z, axis=0)
    truth = {'mu': mu, 'sg_u': sg_u, 'sg_w': sg_w,
             'mu_smp': mu_smp, 'sg_u_smp': sg_u_smp, 'sg_w_smp': sg_w_smp}

    if use_torch:
        data = {
            'x': torch.from_numpy(data['x'].reshape(-1, 1)),
            'y': torch.from_numpy(data['y'].reshape(-1, 1)),
            'r': torch.from_numpy(data['r'].reshape(-1, 1)),
            'u': data['u'],
            'w': data['w']}

    return data, truth

In [ ]:
data, truth = simulate(use_torch=True)
    z_flow, theta_flow, hist_loss = train(
        data,
        n_sample=40,
        epochs=5000)
    z_flow.save("z_flow_artificial")
    theta_flow.save("theta_flow_artificial")
    # z_flow = NormalizingFlow.load("z_flow_artificial")
    theta_flow = NormalizingFlow.load("theta_flow_artificial")
    plot_posterior(theta_flow=theta_flow,
                   truth=truth,
                   name="artificial")
    plot_loss(hist_loss=hist_loss, name="artificial")
